# 応用最適化
## シフト　スケジューリング問題

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from  pulp import *
from ortoolpy import addvars, addbinvars

In [7]:
# データ読み込み
df = pd.read_csv('./abc2.csv')
df
#　　A, B, C, ..., need, week, day 
#  0, 1, 0, ...,     3,          1,  Mon  
#  1, 1, 0, ...,     2,          1,  Tue

,Okabe,Ando,Kato,Yokoyama,Suzuki,Kosaku,need,week,day
0,0,1,1,1,1,0,3,1,MON
1,0,1,1,0,1,0,3,1,TUE
2,1,1,1,1,0,0,3,1,WED
3,1,0,1,1,1,0,3,1,THU
4,1,0,1,1,1,0,3,1,FRI
5,1,1,0,1,0,1,4,1,SUT
6,0,1,0,0,1,1,4,1,SUN
7,1,1,1,1,1,0,3,2,MON
8,1,1,1,1,1,0,3,2,TUE
9,1,0,1,1,0,0,3,2,WED


In [8]:
# 従業員
employee = list(range(len(df.columns)-3))
# 管理者
manager = [1, 2]

n_coma = df.shape[0]
n_emp = df.shape[1]-3
df['vneed'] = addbinvars(n_coma)
v_need_coma = addbinvars(n_emp)
df['vmane'] = addbinvars(n_coma)
employee

[0, 1, 2, 3, 4, 5]

In [10]:
day = np.array(addbinvars(len(df), len(df.columns)-5))
day.shape

(31, 6)

In [11]:
m = pulp.LpProblem() # 最小化
m += (10 * lpSum(df.vneed) 
      + 100 * lpSum(df.apply(lambda r: lpDot(1-r[employee],day[r.name]), 1)) 
      + 1* lpSum(v_need_coma) 
      + 100 * lpSum(df.vmane))

In [12]:
for _,r in df.iterrows():
    m += r.vneed >=  (lpSum(day[r.name]) - r.need)
    m += r.vneed >= -(lpSum(day[r.name]) - r.need)
    m += lpSum(day[r.name, manager]) + r.vmane >= 1

for j,n in enumerate((df.iloc[:,manager].sum(0)+1)//2):
    m += lpSum(day[:,j]) + v_need_coma[j] >= n # 希望の半分以上

%time m.solve()
result = np.vectorize(value)(day).astype(int)
df['result'] = [''.join(i*j for i,j in zip(r,df.columns)) for r in result]
print('mokuteki', value(m.objective))
print(df[['week', 'day','result']])


CPU times: user 12.7 ms, sys: 10.7 ms, total: 23.5 ms
Wall time: 120 ms
mokuteki 20.0
    week  day                   result
0      1  MON       KatoYokoyamaSuzuki
1      1  TUE           AndoKatoSuzuki
2      1  WED        OkabeKatoYokoyama
3      1  THU       KatoYokoyamaSuzuki
4      1  FRI       KatoYokoyamaSuzuki
5      1  SUT  OkabeAndoYokoyamaKosaku
6      1  SUN         AndoSuzukiKosaku
7      2  MON       KatoYokoyamaSuzuki
8      2  TUE       KatoYokoyamaSuzuki
9      2  WED        OkabeKatoYokoyama
10     2  THU           AndoKatoSuzuki
11     2  FRI       KatoYokoyamaSuzuki
12     2  SUT  OkabeAndoYokoyamaKosaku
13     2  SUN  OkabeAndoYokoyamaKosaku
14     3  MON        OkabeKatoYokoyama
15     3  TUE        OkabeKatoYokoyama
16     3  WED           AndoKatoSuzuki
17     3  THU          OkabeKatoSuzuki
18     3  FRI       KatoYokoyamaSuzuki
19     3  SUT  OkabeAndoYokoyamaSuzuki
20     3  SUN  OkabeAndoYokoyamaKosaku
21     4  MON       KatoYokoyamaSuzuki
22     4  TUE    